In [ ]:
!pip install tapi-yandex-direct

In [34]:
import datetime as dt
from pprint import pprint 
from tapi_yandex_direct import YandexDirect

In [9]:
# Токен от аккаунта созданного в песочнице. Можете поиграться.
ACCESS_TOKEN = "AgAAAAADJjMuAAOdM9eC6Scipk-So_vi08XBE0I"

In [10]:
api = YandexDirect(
    # Токен доступа.
    access_token=ACCESS_TOKEN, 
    # Не будет повторять запрос, если закончаться баллы.
    retry_if_not_enough_units=False,
    # Сделает несколько запросов, если кол-во идентификаторов 
    # в условиях фильтрации SelectionCriteria будет больше, 
    # чем можно запросить в одном запросе. Работает для метода "get".
    auto_request_generation=True,
    # Если в запросе не будут получены все объекты, сделает еще запросы.
    receive_all_objects=True,
    # Включить песочницу.
    is_sandbox=True,
    # Если вы делаете запросы из под агенсткого аккаунта, 
    # вам нужно указать логин аккаунта для которого будете делать запросы.
    #login="{логин аккаунта Я.Директ}"
)

### Доступные ресурсы API Я.Директ

In [33]:
pprint(dir(api))

['adextensions',
 'adgroups',
 'adimages',
 'ads',
 'agencyclients',
 'audiencetargets',
 'bidmodifiers',
 'bids',
 'campaigns',
 'changes',
 'clients',
 'creatives',
 'dictionaries',
 'dynamicads',
 'keywordbids',
 'keywords',
 'keywordsresearch',
 'leads',
 'negativekeywordsharedsets',
 'retargeting',
 'sitelinks',
 'turbopages',
 'vcards']


### Создание кампании

In [35]:
body = {
    "method": "add",
    "params": {
        "Campaigns": [
            {
                "Name": "MyCampaignTest",
                "StartDate": str(dt.datetime.now().date()),
                "TextCampaign": {
                    "BiddingStrategy": {
                        "Search": {
                            "BiddingStrategyType": "HIGHEST_POSITION"
                        },
                        "Network": {
                            "BiddingStrategyType": "SERVING_OFF"
                        }
                    },
                    "Settings": []
                }
            }
        ]
    }
}
result = api.campaigns().post(data=body)
data = result().transform()
data

[{'Id': 338584}]

In [7]:
campaign_id = data[0]['Id']
campaign_id

338338

### Создание группы

In [10]:
body = {
    "method": "add",
    "params": {
        "AdGroups": [
            {
                "Name": "MyAdGroupTest",
                "CampaignId": campaign_id,
                "RegionIds": [0],
            }
        ]
    }
}
result = api.adgroups().post(data=body)
data = result().transform()
data

[{'Id': 3687128}]

In [11]:
adgroup_id = data[0]['Id']
adgroup_id

3687128

### Создание набора быстрых ссылок

In [109]:
body = {
  "method": "add",
  "params": {
    "SitelinksSets": [{
        "Sitelinks": [
            {
                "Title": "SitelinkTitle",
                "Href": "https://yandex.ru",
                "Description": "SitelinkDescription",
            },{
                "Title": "SitelinkTitle2",
                "Href": "https://yandex.ru",
                "Description": "SitelinkDescription2",
            },
        ]
    }]
  }
}
result = api.sitelinks().post(data=body)
data = result().transform()
data

[{'Id': 371040}]

In [110]:
links_id = data[0]['Id']
links_id

371040

### Создание объявления

In [113]:
body = {
    "method": "add",
    "params": {
        "Ads": [
            {
                "AdGroupId": adgroup_id,
                "TextAd": {
                    "Title": "MyTitleTest",
                    "Text": "MyTextTest",
                    "Mobile": "NO",
                    "Href": "https://yandex.ru",
                    "SitelinkSetId": links_id
                }
            }
        ]
    }
}
result = api.ads().post(data=body)
data = result().transform()
data

[{'Id': 3140143}]

In [14]:
ads_id = data[0]['Id']
ads_id

3139596

### Создание ключевого слова

In [15]:
body = {
    "method": "add",
    "params": {
        "Keywords": [
            {
                "AdGroupId": adgroup_id,
                "Keyword": "Keyword",
                "Bid": 10 * 1000000,
                "ContextBid": 10 * 1000000,
            }
        ]
    }
}
result = api.keywords().post(data=body)
data = result().transform()
data

[{'Id': 7711613,
  'Warnings': [{'Code': 10160,
    'Details': 'Ставка ContextBid не будет применена, так как отключены показы на сети',
    'Message': 'Ставка не будет применена'}]}]

In [16]:
keyword_id = data[0]['Id']
keyword_id

7711613

### Изменение ставки ключевого слова

In [19]:
body = {
  "method": "set",
  "params": {
    "Bids": [{
      "KeywordId": keyword_id,
      "Bid": 15 * 1000000
    }]
  } 
}
result = api.bids().post(data=body)
data = result().transform()
data

[{'KeywordId': 7711613}]

### Добавление минус слов в кампанию

In [21]:
body = {
  "method": "update",
  "params": {
    "Campaigns": [{
      "Id": campaign_id,
      "NegativeKeywords": {"Items": ["минусслово1", "минусслово2"]}
    }]
  } 
}
result = api.campaigns().post(data=body)
data = result().transform()
data

[{'Id': 338294}]

#### Проверим, получив данные кампании

In [9]:
body = {
    "method": "get",
    "params": {
        "SelectionCriteria": {
            "Ids": [campaign_id]
        },
        "FieldNames": [
            "Id",
            "Name", 
            "NegativeKeywords"
        ],
    } 
}
result = api.campaigns().post(data=body)
data = result().transform()
data

[{'Id': 338294,
  'Name': 'MyCampaignTest',
  'NegativeKeywords': {'Items': ['минусслово1', 'минусслово2']}}]

### Получение всех кампаний

In [39]:
r = api.campaigns().post(
    data={
        "method": "get",
        "params": {
            "SelectionCriteria": {},
            "FieldNames": ["Id","Name"],
            "Page": {"Limit": 1},
        },
    }
)
r().transform()

[{'Id': 338151, 'Name': 'Test API Sandbox campaign 1'},
 {'Id': 338152, 'Name': 'Test API Sandbox campaign 2'},
 {'Id': 338153, 'Name': 'Test API Sandbox campaign 3'},
 {'Id': 338185, 'Name': 'MyCampaignTest'},
 {'Id': 338231, 'Name': 'MyCampaignTest'},
 {'Id': 338232, 'Name': 'MyCampaignTest'},
 {'Id': 338247, 'Name': 'MyCampaignTest'},
 {'Id': 338294, 'Name': 'MyCampaignTest'},
 {'Id': 338338, 'Name': 'MyCampaignTest'},
 {'Id': 338584, 'Name': 'MyCampaignTest'}]

In [40]:
pprint(r().data)

[{'result': {'Campaigns': [{'Id': 338151,
                            'Name': 'Test API Sandbox campaign 1'}],
             'LimitedBy': 1}},
 {'result': {'Campaigns': [{'Id': 338152,
                            'Name': 'Test API Sandbox campaign 2'}],
             'LimitedBy': 2}},
 {'result': {'Campaigns': [{'Id': 338153,
                            'Name': 'Test API Sandbox campaign 3'}],
             'LimitedBy': 3}},
 {'result': {'Campaigns': [{'Id': 338185, 'Name': 'MyCampaignTest'}],
             'LimitedBy': 4}},
 {'result': {'Campaigns': [{'Id': 338231, 'Name': 'MyCampaignTest'}],
             'LimitedBy': 5}},
 {'result': {'Campaigns': [{'Id': 338232, 'Name': 'MyCampaignTest'}],
             'LimitedBy': 6}},
 {'result': {'Campaigns': [{'Id': 338247, 'Name': 'MyCampaignTest'}],
             'LimitedBy': 7}},
 {'result': {'Campaigns': [{'Id': 338294, 'Name': 'MyCampaignTest'}],
             'LimitedBy': 8}},
 {'result': {'Campaigns': [{'Id': 338338, 'Name': 'MyCampaignTest'}],

In [16]:
campaign_ids = [i['Id'] for i in r().transform()]
campaign_ids

[338151, 338152, 338153, 338185, 338231, 338232, 338247, 338294, 338338]

### Получение объявлений

In [24]:
r = api.ads().post(
    data={
        "method": "get",
        "params": {
            "SelectionCriteria": {
                "CampaignIds":campaign_ids,
                "Types": ["TEXT_AD"]
            },
            "FieldNames": ["Id", "Type"],
            "TextAdFieldNames": [
                "Title"
            ]
        },
    }
)
r().transform()

[{'Id': 3139231,
  'TextAd': {'Title': 'Test sandbox banner 1'},
  'Type': 'TEXT_AD'},
 {'Id': 3139232,
  'TextAd': {'Title': 'Test sandbox banner 2'},
  'Type': 'TEXT_AD'},
 {'Id': 3139233,
  'TextAd': {'Title': 'Test sandbox banner 3'},
  'Type': 'TEXT_AD'},
 {'Id': 3139234,
  'TextAd': {'Title': 'Test sandbox banner 4'},
  'Type': 'TEXT_AD'},
 {'Id': 3139235,
  'TextAd': {'Title': 'Test sandbox banner 5'},
  'Type': 'TEXT_AD'},
 {'Id': 3139236,
  'TextAd': {'Title': 'Test sandbox banner 1'},
  'Type': 'TEXT_AD'},
 {'Id': 3139237,
  'TextAd': {'Title': 'Test sandbox banner 2'},
  'Type': 'TEXT_AD'},
 {'Id': 3139238,
  'TextAd': {'Title': 'Test sandbox banner 3'},
  'Type': 'TEXT_AD'},
 {'Id': 3139239,
  'TextAd': {'Title': 'Test sandbox banner 4'},
  'Type': 'TEXT_AD'},
 {'Id': 3139240,
  'TextAd': {'Title': 'Test sandbox banner 5'},
  'Type': 'TEXT_AD'},
 {'Id': 3139241,
  'TextAd': {'Title': 'Test sandbox banner 1'},
  'Type': 'TEXT_AD'},
 {'Id': 3139242,
  'TextAd': {'Title': 'Tes

### Получить идентификаторы объектов, которые были изменены

In [18]:
body = {
  "method": "check",
  "params": {
    "CampaignIds": campaign_ids,  
    "Timestamp": "2019-01-01T00:00:00Z",  # Проверить изменения после указанного времени
    "FieldNames": ["CampaignIds","AdIds"]
  } 
}
r = api.changes().post(data=body)
r().transform()

{'Modified': {'AdIds': [3139231,
   3139232,
   3139233,
   3139234,
   3139235,
   3139236,
   3139237,
   3139238,
   3139239,
   3139240,
   3139241,
   3139242,
   3139243,
   3139244,
   3139245,
   3139335,
   3139529,
   3139596,
   3140143],
  'CampaignIds': [338151,
   338152,
   338153,
   338185,
   338231,
   338232,
   338247,
   338294,
   338338]},
 'Timestamp': '2019-09-08T09:26:24Z'}